In [20]:
from pathlib import Path

try:
    BASE_DIR = Path(__file__).resolve().parent
except NameError:
    BASE_DIR = Path.cwd()


print("Notebook exécuté depuis :", BASE_DIR)


PROJECT_ROOT = BASE_DIR.parent           # /app/Model
DATA_DIR     = PROJECT_ROOT / "OriginalData"
OUTPUT_DIR   = PROJECT_ROOT / "ProcessedData"


OUTPUT_DIR.mkdir(exist_ok=True)

print("DATA_DIR  :", DATA_DIR)
print("OUTPUT_DIR:", OUTPUT_DIR)


Notebook exécuté depuis : /app/Model/ETL
DATA_DIR  : /app/Model/OriginalData
OUTPUT_DIR: /app/Model/ProcessedData


In [21]:
# importation des bibliotheques 
import pandas as pd
import numpy as np
import warnings
from datetime import datetime
import os

# configuration de l'affichage
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
warnings.filterwarnings('ignore')

# verification de l'environnement
print(f"pandas version: {pd.__version__}")
print(f"numpy version: {np.__version__}")
print(f"repertoire de travail: {os.getcwd()}")
print(f"fichiers disponibles: {[f for f in os.listdir('.') if f.endswith('.csv')]}")

pandas version: 2.1.4
numpy version: 1.26.2
repertoire de travail: /app/Model/ETL
fichiers disponibles: []


In [22]:
# chargement des donnees
print("chargement des fichiers csv...")
covid_raw = pd.read_csv(DATA_DIR / 'covid_data.csv')
mpox_raw = pd.read_csv(DATA_DIR / 'mpox_data.csv')

# informations generales sur les datasets
print("\ncovid dataset")
print(f"dimensions: {covid_raw.shape}")
print(f"colonnes: {covid_raw.columns.tolist()}")
print(f"types de donnees:\n{covid_raw.dtypes.value_counts()}")

print("\n mpox dataset ")
print(f"dimensions: {mpox_raw.shape}")
print(f"colonnes: {mpox_raw.columns.tolist()}")
print(f"types de donnees:\n{mpox_raw.dtypes.value_counts()}")

# verification rapide des valeurs manquantes
print("\n taux de valeurs manquantes ")
print(f"covid: {covid_raw.isnull().sum().sum() / (covid_raw.shape[0] * covid_raw.shape[1]):.2%}")
print(f"mpox: {mpox_raw.isnull().sum().sum() / (mpox_raw.shape[0] * mpox_raw.shape[1]):.2%}")

chargement des fichiers csv...

covid dataset
dimensions: (429435, 67)
colonnes: ['iso_code', 'continent', 'location', 'date', 'total_cases', 'new_cases', 'new_cases_smoothed', 'total_deaths', 'new_deaths', 'new_deaths_smoothed', 'total_cases_per_million', 'new_cases_per_million', 'new_cases_smoothed_per_million', 'total_deaths_per_million', 'new_deaths_per_million', 'new_deaths_smoothed_per_million', 'reproduction_rate', 'icu_patients', 'icu_patients_per_million', 'hosp_patients', 'hosp_patients_per_million', 'weekly_icu_admissions', 'weekly_icu_admissions_per_million', 'weekly_hosp_admissions', 'weekly_hosp_admissions_per_million', 'total_tests', 'new_tests', 'total_tests_per_thousand', 'new_tests_per_thousand', 'new_tests_smoothed', 'new_tests_smoothed_per_thousand', 'positive_rate', 'tests_per_case', 'tests_units', 'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'total_boosters', 'new_vaccinations', 'new_vaccinations_smoothed', 'total_vaccinations_per_hundred'

In [23]:
# analyse detaillee de la structure des donnees
print(" analyse des dates")
print(f"covid - type date: {covid_raw['date'].dtype}")
print(f"covid - exemples dates: {covid_raw['date'].head(3).tolist()}")
print(f"covid - plage dates: {covid_raw['date'].min()} a {covid_raw['date'].max()}")

print(f"\nmpox - type date: {mpox_raw['date'].dtype}")
print(f"mpox - exemples dates: {mpox_raw['date'].head(3).tolist()}")
print(f"mpox - plage dates: {mpox_raw['date'].min()} a {mpox_raw['date'].max()}")

# analyse des locations et agregations
print("\n analyse des locations")
covid_locations = covid_raw['location'].unique()
mpox_locations = mpox_raw['location'].unique()

print(f"covid - nombre locations uniques: {len(covid_locations)}")
print(f"mpox - nombre locations uniques: {len(mpox_locations)}")

# identification des agregations potentielles
agregations_patterns = ['World', 'Europe', 'Asia', 'Africa', 'America', 'Oceania', 'income', 'Union']
covid_agregations = [loc for loc in covid_locations if any(pattern in loc for pattern in agregations_patterns)]
mpox_agregations = [loc for loc in mpox_locations if any(pattern in loc for pattern in agregations_patterns)]

print(f"\ncovid - agregations detectees ({len(covid_agregations)}): {sorted(covid_agregations)}")
print(f"\nmpox - agregations detectees ({len(mpox_agregations)}): {sorted(mpox_agregations)}")

# verification des colonnes communes
colonnes_communes = set(covid_raw.columns) & set(mpox_raw.columns)
print(f"\n colonnes communes ({len(colonnes_communes)})")
print(sorted(colonnes_communes))

 analyse des dates
covid - type date: object
covid - exemples dates: ['2020-01-05', '2020-01-06', '2020-01-07']
covid - plage dates: 2020-01-01 a 2024-08-14

mpox - type date: object
mpox - exemples dates: ['2022-05-01', '2022-05-02', '2022-05-03']
mpox - plage dates: 2022-05-01 a 2025-05-31

 analyse des locations
covid - nombre locations uniques: 255
mpox - nombre locations uniques: 144

covid - agregations detectees (15): ['Africa', 'American Samoa', 'Asia', 'Central African Republic', 'Europe', 'European Union (27)', 'High-income countries', 'Low-income countries', 'Lower-middle-income countries', 'North America', 'Oceania', 'South Africa', 'South America', 'Upper-middle-income countries', 'World']

mpox - agregations detectees (9): ['Africa', 'Asia', 'Central African Republic', 'Europe', 'North America', 'Oceania', 'South Africa', 'South America', 'World']

 colonnes communes (15)
['date', 'iso_code', 'location', 'new_cases', 'new_cases_per_million', 'new_cases_smoothed', 'new_cas

In [24]:
# conversion des dates en datetime
print("conversion des dates en datetime...")
covid_raw['date'] = pd.to_datetime(covid_raw['date'])
mpox_raw['date'] = pd.to_datetime(mpox_raw['date'])

print("conversion terminee")
print(f"covid - type date: {covid_raw['date'].dtype}")
print(f"mpox - type date: {mpox_raw['date'].dtype}")

# affinement de la detection des agregations (exclure les pays)
agregations_exactes = [
    'World', 'Africa', 'Asia', 'Europe', 'North America', 'South America', 'Oceania',
    'European Union', 'High-income countries', 'Low-income countries', 
    'Lower-middle-income countries', 'Upper-middle-income countries'
]

covid_agregations_reelles = []
mpox_agregations_reelles = []

for loc in covid_raw['location'].unique():
    if any(ag in loc for ag in agregations_exactes):
        # exclure les pays qui contiennent ces mots
        if loc not in ['Central African Republic', 'South Africa', 'American Samoa']:
            covid_agregations_reelles.append(loc)

for loc in mpox_raw['location'].unique():
    if any(ag in loc for ag in agregations_exactes):
        # exclure les pays qui contiennent ces mots
        if loc not in ['Central African Republic', 'South Africa']:
            mpox_agregations_reelles.append(loc)

print(f"\n agregations reelles ")
print(f"covid ({len(covid_agregations_reelles)}): {sorted(covid_agregations_reelles)}")
print(f"mpox ({len(mpox_agregations_reelles)}): {sorted(mpox_agregations_reelles)}")

# verification de la presence de doublons avant traitement
print("\n verification des doublons (location, date) ")
covid_doublons = covid_raw[covid_raw.duplicated(subset=['location', 'date'], keep=False)]
mpox_doublons = mpox_raw[mpox_raw.duplicated(subset=['location', 'date'], keep=False)]

print(f"covid - doublons: {len(covid_doublons)} lignes ({len(covid_doublons)//2} paires)")
print(f"mpox - doublons: {len(mpox_doublons)} lignes ({len(mpox_doublons)//2} paires)")

conversion des dates en datetime...
conversion terminee
covid - type date: datetime64[ns]
mpox - type date: datetime64[ns]

 agregations reelles 
covid (12): ['Africa', 'Asia', 'Europe', 'European Union (27)', 'High-income countries', 'Low-income countries', 'Lower-middle-income countries', 'North America', 'Oceania', 'South America', 'Upper-middle-income countries', 'World']
mpox (7): ['Africa', 'Asia', 'Europe', 'North America', 'Oceania', 'South America', 'World']

 verification des doublons (location, date) 
covid - doublons: 15540 lignes (7770 paires)
mpox - doublons: 0 lignes (0 paires)


In [25]:
# analyse detaillee des doublons covid
print(" analyse des doublons covid ")
if len(covid_doublons) > 0:
    # exemple de doublons pour comprendre le pattern
    exemple_location = covid_doublons['location'].iloc[0]
    exemple_date = covid_doublons['date'].iloc[0]
    
    exemple_doublons = covid_raw[(covid_raw['location'] == exemple_location) & 
                                 (covid_raw['date'] == exemple_date)]
    
    print(f"exemple de doublons pour {exemple_location} le {exemple_date}:")
    colonnes_interessantes = ['location', 'date', 'new_cases', 'total_cases', 'new_deaths', 'total_deaths']
    print(exemple_doublons[colonnes_interessantes])
    
    # analyse du pattern des doublons
    print("\n pattern des doublons ")
    # compter les valeurs non-nan pour chaque ligne dupliquee
    covid_doublons_copy = covid_doublons.copy()
    colonnes_numeriques = covid_raw.select_dtypes(include=[np.number]).columns
    covid_doublons_copy['nb_valeurs_non_nan'] = covid_doublons_copy[colonnes_numeriques].notna().sum(axis=1)
    
    # grouper par location/date et voir la distribution
    pattern_doublons = covid_doublons_copy.groupby(['location', 'date'])['nb_valeurs_non_nan'].agg(['count', 'min', 'max'])
    print(f"tous les doublons ont exactement 2 lignes: {(pattern_doublons['count'] == 2).all()}")
    print(f"pattern type - une ligne avec donnees, une avec peu/pas de donnees: {(pattern_doublons['min'] < pattern_doublons['max']).sum()} / {len(pattern_doublons)}")
    
    # verification que les doublons concernent bien des lignes complementaires
    sample_pattern = pattern_doublons.head(5)
    print("\nexemples de patterns (nb valeurs non-nan min/max par paire):")
    print(sample_pattern)

 analyse des doublons covid 
exemple de doublons pour East Timor le 2021-02-12 00:00:00:
          location       date  new_cases  total_cases  new_deaths  \
102540  East Timor 2021-02-12      0.000       80.000       0.000   
380675  East Timor 2021-02-12        NaN          NaN         NaN   

        total_deaths  
102540         0.000  
380675           NaN  

 pattern des doublons 
tous les doublons ont exactement 2 lignes: True
pattern type - une ligne avec donnees, une avec peu/pas de donnees: 7660 / 7770

exemples de patterns (nb valeurs non-nan min/max par paire):
                       count  min  max
location   date                       
East Timor 2021-02-12      2   17   27
           2021-02-13      2   17   27
           2021-02-14      2   17   27
           2021-02-15      2   17   27
           2021-02-16      2   17   27


In [26]:
# fonction pour fusionner les lignes dupliquees
def merge_duplicate_rows(df, subset=['location', 'date']):
    """
    fusionne les lignes dupliquees en gardant les valeurs non-nan
    """
    print(f"traitement des doublons sur {subset}...")
    
    # identifier les doublons
    duplicated_mask = df.duplicated(subset=subset, keep=False)
    df_no_dup = df[~duplicated_mask].copy()
    df_dup = df[duplicated_mask].copy()
    
    if len(df_dup) == 0:
        print("aucun doublon trouve")
        return df
    
    print(f"doublons trouves: {len(df_dup)} lignes")
    
    # pour chaque groupe de doublons, garder les valeurs non-nan
    def merge_group(group):
        # pour les colonnes numeriques, prendre la premiere valeur non-nan
        numeric_cols = group.select_dtypes(include=[np.number]).columns
        result = group.iloc[0].copy()
        
        for col in numeric_cols:
            non_nan_values = group[col].dropna()
            if len(non_nan_values) > 0:
                result[col] = non_nan_values.iloc[0]
        
        return result
    
    # appliquer la fusion
    df_merged = df_dup.groupby(subset).apply(merge_group).reset_index(drop=True)
    
    # combiner avec les non-doublons
    df_final = pd.concat([df_no_dup, df_merged], ignore_index=True)
    
    print(f"resultat: {len(df)} lignes -> {len(df_final)} lignes")
    print(f"lignes supprimees: {len(df) - len(df_final)}")
    
    return df_final

# application aux datasets
print(" traitement des doublons ")
covid_dedup = merge_duplicate_rows(covid_raw, subset=['location', 'date'])
mpox_dedup = merge_duplicate_rows(mpox_raw, subset=['location', 'date'])

# verification
print("\n verification apres deduplication ")
covid_doublons_apres = covid_dedup[covid_dedup.duplicated(subset=['location', 'date'], keep=False)]
mpox_doublons_apres = mpox_dedup[mpox_dedup.duplicated(subset=['location', 'date'], keep=False)]
print(f"covid - doublons restants: {len(covid_doublons_apres)}")
print(f"mpox - doublons restants: {len(mpox_doublons_apres)}")

 traitement des doublons 
traitement des doublons sur ['location', 'date']...
doublons trouves: 15540 lignes
resultat: 429435 lignes -> 421665 lignes
lignes supprimees: 7770
traitement des doublons sur ['location', 'date']...
aucun doublon trouve

 verification apres deduplication 
covid - doublons restants: 0
mpox - doublons restants: 0


In [27]:
# analyse des valeurs cumulatives decroissantes
print(" analyse des valeurs cumulatives decroissantes ")

def check_cumulative_decrease(df, col, location_col='location'):
    """
    verifie si une colonne cumulative decroit
    """
    if col not in df.columns:
        return None
    
    # trier par location et date
    df_sorted = df.sort_values([location_col, 'date'])
    
    # calculer les differences pour chaque location
    decreases = []
    for location in df_sorted[location_col].unique():
        loc_data = df_sorted[df_sorted[location_col] == location]
        # ignorer les nan
        values = loc_data[col].dropna()
        if len(values) > 1:
            diffs = values.diff()
            negative_diffs = diffs[diffs < 0]
            if len(negative_diffs) > 0:
                for idx in negative_diffs.index:
                    row = loc_data.loc[idx]
                    prev_idx = values.index[values.index.get_loc(idx) - 1]
                    prev_val = values.loc[prev_idx]
                    decreases.append({
                        'location': location,
                        'date': row['date'],
                        'valeur_actuelle': values.loc[idx],
                        'valeur_precedente': prev_val,
                        'difference': negative_diffs.loc[idx]
                    })
    
    return pd.DataFrame(decreases)

# verification pour covid
print("\ncovid - verification total_cases:")
covid_cases_decreases = check_cumulative_decrease(covid_dedup, 'total_cases')
if covid_cases_decreases is not None and len(covid_cases_decreases) > 0:
    print(f"cas de decroissance: {len(covid_cases_decreases)}")
    print("exemples:")
    print(covid_cases_decreases.head(3))

print("\ncovid - verification total_deaths:")
covid_deaths_decreases = check_cumulative_decrease(covid_dedup, 'total_deaths')
if covid_deaths_decreases is not None and len(covid_deaths_decreases) > 0:
    print(f"cas de decroissance: {len(covid_deaths_decreases)}")
    print("exemples:")
    print(covid_deaths_decreases.head(3))

# verification pour mpox
print("\nmpox - verification total_cases:")
mpox_cases_decreases = check_cumulative_decrease(mpox_dedup, 'total_cases')
if mpox_cases_decreases is not None and len(mpox_cases_decreases) > 0:
    print(f"cas de decroissance: {len(mpox_cases_decreases)}")

print("\nmpox - verification total_deaths:")
mpox_deaths_decreases = check_cumulative_decrease(mpox_dedup, 'total_deaths')
if mpox_deaths_decreases is not None and len(mpox_deaths_decreases) > 0:
    print(f"cas de decroissance: {len(mpox_deaths_decreases)}")

 analyse des valeurs cumulatives decroissantes 

covid - verification total_cases:
cas de decroissance: 20
exemples:
  location       date  valeur_actuelle  valeur_precedente  difference
0   Belize 2022-04-10        57287.000          57289.000      -2.000
1  Burundi 2023-07-16        54216.000          54321.000    -105.000
2  Ecuador 2020-09-13       116451.000         118045.000   -1594.000

covid - verification total_deaths:
cas de decroissance: 15
exemples:
                          location       date  valeur_actuelle  \
0                        Australia 2023-07-23        22694.000   
1  Bonaire Sint Eustatius and Saba 2021-10-17           19.000   
2                           Canada 2022-06-19        41017.000   

   valeur_precedente  difference  
0          22770.000     -76.000  
1             20.000      -1.000  
2          41348.000    -331.000  

mpox - verification total_cases:

mpox - verification total_deaths:


In [28]:
# correction des valeurs cumulatives decroissantes
print(" correction des valeurs cumulatives ")

def fix_cumulative_values(df, cumulative_cols, location_col='location'):
    """
    corrige les valeurs cumulatives avec cummax par location
    """
    df_fixed = df.copy()
    
    for col in cumulative_cols:
        if col in df_fixed.columns:
            print(f"correction de {col}...")
            # trier par location et date
            df_fixed = df_fixed.sort_values([location_col, 'date'])
            # appliquer cummax par location
            df_fixed[col] = df_fixed.groupby(location_col)[col].transform(lambda x: x.cummax())
    
    return df_fixed

# colonnes cumulatives a corriger
cumulative_columns = ['total_cases', 'total_deaths', 'total_tests', 'total_vaccinations', 
                     'people_vaccinated', 'people_fully_vaccinated', 'total_boosters']

# application des corrections
covid_fixed = fix_cumulative_values(covid_dedup, cumulative_columns)
mpox_fixed = fix_cumulative_values(mpox_dedup, ['total_cases', 'total_deaths'])

# verification apres correction
print("\n verification apres correction ")
print("covid - total_cases:")
covid_cases_decreases_after = check_cumulative_decrease(covid_fixed, 'total_cases')
print(f"cas de decroissance restants: {len(covid_cases_decreases_after) if covid_cases_decreases_after is not None else 0}")

print("\ncovid - total_deaths:")
covid_deaths_decreases_after = check_cumulative_decrease(covid_fixed, 'total_deaths')
print(f"cas de decroissance restants: {len(covid_deaths_decreases_after) if covid_deaths_decreases_after is not None else 0}")

# verification que les valeurs ont bien ete corrigees
print("\n exemple de correction ")
# prendre un exemple de correction
if len(covid_cases_decreases) > 0:
    exemple = covid_cases_decreases.iloc[0]
    loc = exemple['location']
    date = exemple['date']
    
    # valeurs avant/apres
    avant = covid_dedup[(covid_dedup['location'] == loc) & (covid_dedup['date'] == date)]['total_cases'].iloc[0]
    apres = covid_fixed[(covid_fixed['location'] == loc) & (covid_fixed['date'] == date)]['total_cases'].iloc[0]
    
    print(f"exemple pour {loc} le {date}:")
    print(f"valeur avant correction: {avant}")
    print(f"valeur apres correction: {apres}")

 correction des valeurs cumulatives 
correction de total_cases...
correction de total_deaths...
correction de total_tests...
correction de total_vaccinations...
correction de people_vaccinated...
correction de people_fully_vaccinated...
correction de total_boosters...
correction de total_cases...
correction de total_deaths...

 verification apres correction 
covid - total_cases:
cas de decroissance restants: 0

covid - total_deaths:
cas de decroissance restants: 0

 exemple de correction 
exemple pour Belize le 2022-04-10 00:00:00:
valeur avant correction: 57287.0
valeur apres correction: 57289.0


In [29]:
# analyse des lignes avec toutes valeurs numeriques nan
print(" analyse des lignes toutes nan ")

def find_all_nan_rows(df):
    """
    trouve les lignes ou toutes les colonnes numeriques sont nan
    """
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    all_nan_mask = df[numeric_cols].isna().all(axis=1)
    return df[all_nan_mask]

# recherche dans covid
covid_all_nan = find_all_nan_rows(covid_fixed)
print(f"covid - lignes avec toutes valeurs numeriques nan: {len(covid_all_nan)}")
if len(covid_all_nan) > 0:
    print("exemples:")
    print(covid_all_nan[['location', 'date', 'total_cases', 'new_cases', 'total_deaths']].head())

# recherche dans mpox
mpox_all_nan = find_all_nan_rows(mpox_fixed)
print(f"\nmpox - lignes avec toutes valeurs numeriques nan: {len(mpox_all_nan)}")
if len(mpox_all_nan) > 0:
    print("exemples:")
    print(mpox_all_nan[['location', 'date', 'total_cases', 'new_cases', 'total_deaths']].head())
    print(f"locations concernees: {mpox_all_nan['location'].unique()}")

# suppression des lignes toutes nan
print("\n suppression des lignes toutes nan ")
covid_cleaned = covid_fixed[~covid_fixed.index.isin(covid_all_nan.index)]
mpox_cleaned = mpox_fixed[~mpox_fixed.index.isin(mpox_all_nan.index)]

print(f"covid: {len(covid_fixed)} -> {len(covid_cleaned)} lignes ({len(covid_all_nan)} supprimees)")
print(f"mpox: {len(mpox_fixed)} -> {len(mpox_cleaned)} lignes ({len(mpox_all_nan)} supprimees)")

 analyse des lignes toutes nan 
covid - lignes avec toutes valeurs numeriques nan: 0

mpox - lignes avec toutes valeurs numeriques nan: 1
exemples:
      location       date  total_cases  new_cases  total_deaths
18272  Burundi 2024-07-19          NaN        NaN           NaN
locations concernees: ['Burundi']

 suppression des lignes toutes nan 
covid: 421665 -> 421665 lignes (0 supprimees)
mpox: 123852 -> 123851 lignes (1 supprimees)


In [30]:
# analyse des valeurs negatives
print(" analyse des valeurs negatives ")

# fonction pour analyser les valeurs negatives
def analyze_negative_values(df, columns_to_check):
    """
    analyse les valeurs negatives dans les colonnes specifiees
    """
    results = {}
    for col in columns_to_check:
        if col in df.columns:
            negative_mask = df[col] < 0
            negative_values = df[negative_mask]
            if len(negative_values) > 0:
                results[col] = {
                    'count': len(negative_values),
                    'min': df[col].min(),
                    'max': negative_values[col].max(),
                    'locations': len(negative_values['location'].unique())
                }
    return results

# colonnes a verifier pour valeurs negatives
cols_to_check = ['reproduction_rate', 'new_cases', 'new_deaths', 'total_cases', 'total_deaths']

# analyse covid
print("covid - valeurs negatives:")
covid_negatives = analyze_negative_values(covid_cleaned, cols_to_check)
for col, stats in covid_negatives.items():
    print(f"  {col}: {stats['count']} valeurs, min={stats['min']:.3f}, max={stats['max']:.3f}, {stats['locations']} locations")

# analyse mpox
print("\nmpox - valeurs negatives:")
mpox_negatives = analyze_negative_values(mpox_cleaned, cols_to_check)
for col, stats in mpox_negatives.items():
    print(f"  {col}: {stats['count']} valeurs, min={stats['min']:.3f}, max={stats['max']:.3f}, {stats['locations']} locations")

# exemples de reproduction_rate negatif
if 'reproduction_rate' in covid_negatives:
    print("\n exemples de reproduction_rate negatif ")
    negative_repro = covid_cleaned[covid_cleaned['reproduction_rate'] < 0]
    print(negative_repro[['location', 'date', 'reproduction_rate', 'new_cases', 'total_cases']].head())

 analyse des valeurs negatives 
covid - valeurs negatives:
  reproduction_rate: 178 valeurs, min=-0.070, max=-0.010, 14 locations

mpox - valeurs negatives:

 exemples de reproduction_rate negatif 
      location       date  reproduction_rate  new_cases  total_cases
11062   Angola 2022-10-19             -0.010      0.000   103299.000
11069   Angola 2022-10-26             -0.010      0.000   103414.000
17823  Armenia 2022-12-19             -0.070      0.000   445525.000
17836  Armenia 2023-01-01             -0.010     37.000   445620.000
17837  Armenia 2023-01-02             -0.070      0.000   445620.000


In [31]:
# correction des valeurs negatives
print(" correction des valeurs negatives ")

# correction du reproduction_rate negatif
if 'reproduction_rate' in covid_cleaned.columns:
    print("correction du reproduction_rate negatif...")
    mask_negative = covid_cleaned['reproduction_rate'] < 0
    nb_corrections = mask_negative.sum()
    covid_cleaned.loc[mask_negative, 'reproduction_rate'] = 0
    print(f"reproduction_rate: {nb_corrections} valeurs negatives remplacees par 0")
    
    # verification
    nb_negative_after = (covid_cleaned['reproduction_rate'] < 0).sum()
    print(f"valeurs negatives restantes: {nb_negative_after}")

# analyse du mortality_rate
print("\n analyse du mortality_rate ")
print("le mortality_rate n'existe pas dans les donnees, il faudra le calculer")
print("formule: mortality_rate = (total_deaths / total_cases) * 100")

# verification de la presence de mortality_rate
print(f"\ncovid - 'mortality_rate' present: {'mortality_rate' in covid_cleaned.columns}")
print(f"mpox - 'mortality_rate' present: {'mortality_rate' in mpox_cleaned.columns}")

# calcul preliminaire pour voir les problemes potentiels
print("\n test du calcul mortality_rate ")
# test sur un echantillon
covid_sample = covid_cleaned[covid_cleaned['total_deaths'].notna() & covid_cleaned['total_cases'].notna()].head(1000)
if len(covid_sample) > 0:
    test_mortality = (covid_sample['total_deaths'] / covid_sample['total_cases']) * 100
    print(f"valeurs > 100%: {(test_mortality > 100).sum()}")
    print(f"valeurs infinies: {np.isinf(test_mortality).sum()}")
    print(f"valeurs nan (0/0): {test_mortality.isna().sum()}")
    
    if (test_mortality > 100).any():
        print("\nexemples de mortality_rate > 100%:")
        problematic = covid_sample[test_mortality > 100][['location', 'date', 'total_deaths', 'total_cases']]
        print(problematic.head(3))

 correction des valeurs negatives 
correction du reproduction_rate negatif...
reproduction_rate: 178 valeurs negatives remplacees par 0
valeurs negatives restantes: 0

 analyse du mortality_rate 
le mortality_rate n'existe pas dans les donnees, il faudra le calculer
formule: mortality_rate = (total_deaths / total_cases) * 100

covid - 'mortality_rate' present: False
mpox - 'mortality_rate' present: False

 test du calcul mortality_rate 
valeurs > 100%: 0
valeurs infinies: 0
valeurs nan (0/0): 56


In [32]:
# separation pays vs agregations
print(" separation pays vs agregations ")

# listes des agregations identifiees precedemment
covid_agregations_list = ['Africa', 'Asia', 'Europe', 'European Union (27)', 
                         'High-income countries', 'Low-income countries', 
                         'Lower-middle-income countries', 'North America', 
                         'Oceania', 'South America', 'Upper-middle-income countries', 'World']

mpox_agregations_list = ['Africa', 'Asia', 'Europe', 'North America', 
                        'Oceania', 'South America', 'World']

# separation covid
covid_countries = covid_cleaned[~covid_cleaned['location'].isin(covid_agregations_list)]
covid_aggregations = covid_cleaned[covid_cleaned['location'].isin(covid_agregations_list)]

print(f"covid - pays: {len(covid_countries)} lignes, {covid_countries['location'].nunique()} locations")
print(f"covid - agregations: {len(covid_aggregations)} lignes, {covid_aggregations['location'].nunique()} locations")

# separation mpox
mpox_countries = mpox_cleaned[~mpox_cleaned['location'].isin(mpox_agregations_list)]
mpox_aggregations = mpox_cleaned[mpox_cleaned['location'].isin(mpox_agregations_list)]

print(f"\nmpox - pays: {len(mpox_countries)} lignes, {mpox_countries['location'].nunique()} locations")
print(f"mpox - agregations: {len(mpox_aggregations)} lignes, {mpox_aggregations['location'].nunique()} locations")

# verification des locations
print("\n verification des separations ")
print(f"covid - locations agregations: {sorted(covid_aggregations['location'].unique())}")
print(f"\nmpox - locations agregations: {sorted(mpox_aggregations['location'].unique())}")

# verification qu'aucun pays n'a ete mal classe
pays_suspects = ['South Africa', 'Central African Republic', 'American Samoa']
for pays in pays_suspects:
    if pays in covid_agregations_list or pays in mpox_agregations_list:
        print(f"attention: {pays} classe comme agregation!")
    else:
        in_covid = pays in covid_countries['location'].values
        in_mpox = pays in mpox_countries['location'].values
        print(f"{pays} - dans covid pays: {in_covid}, dans mpox pays: {in_mpox}")

 separation pays vs agregations 
covid - pays: 401502 lignes, 243 locations
covid - agregations: 20163 lignes, 12 locations

mpox - pays: 116057 lignes, 137 locations
mpox - agregations: 7794 lignes, 7 locations

 verification des separations 
covid - locations agregations: ['Africa', 'Asia', 'Europe', 'European Union (27)', 'High-income countries', 'Low-income countries', 'Lower-middle-income countries', 'North America', 'Oceania', 'South America', 'Upper-middle-income countries', 'World']

mpox - locations agregations: ['Africa', 'Asia', 'Europe', 'North America', 'Oceania', 'South America', 'World']
South Africa - dans covid pays: True, dans mpox pays: True
Central African Republic - dans covid pays: True, dans mpox pays: True
American Samoa - dans covid pays: True, dans mpox pays: False


In [33]:
# enrichissement de mpox avec donnees statiques de covid
print(" enrichissement de mpox avec donnees covid ")

# colonnes statiques a recuperer de covid (ne changent pas avec le temps)
colonnes_statiques = ['population', 'population_density', 'median_age', 'aged_65_older', 
                     'aged_70_older', 'gdp_per_capita', 'extreme_poverty', 
                     'cardiovasc_death_rate', 'diabetes_prevalence', 'female_smokers', 
                     'male_smokers', 'handwashing_facilities', 'hospital_beds_per_thousand', 
                     'life_expectancy', 'human_development_index']

# pour chaque pays, recuperer les valeurs statiques les plus recentes de covid
print("extraction des donnees statiques de covid...")
covid_static_data = {}

for location in covid_countries['location'].unique():
    loc_data = covid_countries[covid_countries['location'] == location]
    # prendre la ligne la plus recente
    latest_data = loc_data.sort_values('date', ascending=False).iloc[0]
    
    static_values = {}
    for col in colonnes_statiques:
        if col in latest_data.index:
            value = latest_data[col]
            # garder seulement si non nan
            if pd.notna(value):
                static_values[col] = value
    
    if static_values:
        covid_static_data[location] = static_values

print(f"donnees statiques extraites pour {len(covid_static_data)} pays")

# enrichir mpox_countries
print("\nenrichissement de mpox...")
mpox_enriched = mpox_countries.copy()

# ajouter les colonnes si elles n'existent pas
for col in colonnes_statiques:
    if col not in mpox_enriched.columns:
        mpox_enriched[col] = np.nan

# remplir avec les donnees statiques
nb_enrichissements = 0
for idx, row in mpox_enriched.iterrows():
    location = row['location']
    if location in covid_static_data:
        for col, value in covid_static_data[location].items():
            if pd.isna(mpox_enriched.at[idx, col]):
                mpox_enriched.at[idx, col] = value
                nb_enrichissements += 1

print(f"nombre de valeurs ajoutees: {nb_enrichissements}")

# verification de l'enrichissement
print("\n verification de l'enrichissement ")
for col in ['population', 'population_density']:
    avant = mpox_countries[col].notna().sum() if col in mpox_countries.columns else 0
    apres = mpox_enriched[col].notna().sum()
    print(f"{col}: {avant} -> {apres} valeurs non-nan")

# exemple d'enrichissement
exemple_location = mpox_enriched['location'].iloc[0]
print(f"\nexemple pour {exemple_location}:")
print(f"population: {mpox_enriched[mpox_enriched['location'] == exemple_location]['population'].iloc[0]}")
print(f"population_density: {mpox_enriched[mpox_enriched['location'] == exemple_location]['population_density'].iloc[0]}")

 enrichissement de mpox avec donnees covid 
extraction des donnees statiques de covid...
donnees statiques extraites pour 243 pays

enrichissement de mpox...
nombre de valeurs ajoutees: 1502068

 verification de l'enrichissement 
population: 0 -> 116057 valeurs non-nan
population_density: 0 -> 113870 valeurs non-nan

exemple pour Albania:
population: 2842318.0
population_density: 104.87


In [34]:
# calcul du mortality_rate
print(" calcul du mortality_rate ")

def calculate_mortality_rate(df):
    """
    calcule le mortality_rate de facon robuste
    """
    df_copy = df.copy()
    
    # calcul de base
    df_copy['mortality_rate'] = (df_copy['total_deaths'] / df_copy['total_cases']) * 100
    
    # gestion des cas speciaux
    # cas ou total_cases = 0 (division par zero -> inf)
    mask_zero_cases = df_copy['total_cases'] == 0
    df_copy.loc[mask_zero_cases, 'mortality_rate'] = np.nan
    
    # cas ou mortality_rate > 100% (erreur de donnees)
    mask_over_100 = df_copy['mortality_rate'] > 100
    if mask_over_100.sum() > 0:
        print(f"valeurs > 100% detectees: {mask_over_100.sum()}")
        df_copy.loc[mask_over_100, 'mortality_rate'] = 100.0
    
    # remplacer les inf par nan
    df_copy['mortality_rate'] = df_copy['mortality_rate'].replace([np.inf, -np.inf], np.nan)
    
    return df_copy

# application aux deux datasets
print("calcul pour covid...")
covid_countries = calculate_mortality_rate(covid_countries)
covid_aggregations = calculate_mortality_rate(covid_aggregations)

print("\ncalcul pour mpox...")
mpox_enriched = calculate_mortality_rate(mpox_enriched)
mpox_aggregations = calculate_mortality_rate(mpox_aggregations)

# verification
print("\n verification mortality_rate ")
for name, df in [('covid_countries', covid_countries), ('mpox_enriched', mpox_enriched)]:
    if 'mortality_rate' in df.columns:
        stats = df['mortality_rate'].describe()
        print(f"\n{name}:")
        print(f"  min: {stats['min']:.3f}")
        print(f"  max: {stats['max']:.3f}")
        print(f"  moyenne: {stats['mean']:.3f}")
        print(f"  valeurs non-nan: {df['mortality_rate'].notna().sum()}")

 calcul du mortality_rate 
calcul pour covid...
valeurs > 100% detectees: 196
valeurs > 100% detectees: 49

calcul pour mpox...

 verification mortality_rate 

covid_countries:
  min: 0.000
  max: 100.000
  moyenne: 1.776
  valeurs non-nan: 362629

mpox_enriched:
  min: 0.000
  max: 100.000
  moyenne: 1.227
  valeurs non-nan: 115281


In [35]:
# selection des colonnes finales
print(" selection des colonnes finales ")

# definition des colonnes essentielles
colonnes_identifiants = ['iso_code', 'continent', 'location', 'date']
colonnes_transmission = ['new_cases', 'total_cases', 'new_cases_smoothed', 
                        'new_cases_per_million', 'new_cases_smoothed_per_million', 
                        'reproduction_rate']
colonnes_mortalite = ['new_deaths', 'total_deaths', 'new_deaths_smoothed', 
                     'new_deaths_per_million', 'new_deaths_smoothed_per_million', 
                     'total_deaths_per_million', 'mortality_rate']
colonnes_population = ['population', 'population_density']
colonnes_politique = ['stringency_index']

# toutes les colonnes a garder
colonnes_finales = (colonnes_identifiants + colonnes_transmission + 
                   colonnes_mortalite + colonnes_population + colonnes_politique)

print(f"nombre total de colonnes a garder: {len(colonnes_finales)}")

# verification de la disponibilite des colonnes
print("\n verification disponibilite colonnes ")
for dataset_name, dataset in [('covid_countries', covid_countries), 
                              ('mpox_enriched', mpox_enriched)]:
    print(f"\n{dataset_name}:")
    colonnes_manquantes = [col for col in colonnes_finales if col not in dataset.columns]
    colonnes_presentes = [col for col in colonnes_finales if col in dataset.columns]
    
    print(f"  colonnes presentes: {len(colonnes_presentes)}/{len(colonnes_finales)}")
    if colonnes_manquantes:
        print(f"  colonnes manquantes: {colonnes_manquantes}")
    
    # taux de remplissage pour les colonnes a tester
    for col in ['population_density', 'stringency_index']:
        if col in dataset.columns:
            taux = dataset[col].notna().sum() / len(dataset) * 100
            print(f"  {col}: {taux:.1f}% de valeurs non-nan")

# selection des colonnes disponibles
covid_countries_final = covid_countries[[col for col in colonnes_finales if col in covid_countries.columns]]
mpox_enriched_final = mpox_enriched[[col for col in colonnes_finales if col in mpox_enriched.columns]]

print(f"\n datasets finaux ")
print(f"covid_countries: {covid_countries_final.shape}")
print(f"mpox_enriched: {mpox_enriched_final.shape}")

 selection des colonnes finales 
nombre total de colonnes a garder: 20

 verification disponibilite colonnes 

covid_countries:
  colonnes presentes: 20/20
  population_density: 89.0% de valeurs non-nan
  stringency_index: 48.9% de valeurs non-nan

mpox_enriched:
  colonnes presentes: 17/20
  colonnes manquantes: ['continent', 'reproduction_rate', 'stringency_index']
  population_density: 98.1% de valeurs non-nan

 datasets finaux 
covid_countries: (401502, 20)
mpox_enriched: (116057, 17)


In [36]:
# enrichissement des colonnes manquantes dans mpox
print(" enrichissement des colonnes manquantes dans mpox ")

# enrichir continent depuis covid
print("ajout de la colonne continent...")
continent_mapping = {}
for location in covid_countries['location'].unique():
    continent = covid_countries[covid_countries['location'] == location]['continent'].iloc[0]
    if pd.notna(continent):
        continent_mapping[location] = continent

mpox_enriched_final['continent'] = mpox_enriched_final['location'].map(continent_mapping)
print(f"continent ajoute pour {mpox_enriched_final['continent'].notna().sum()} lignes")

# stringency_index - donnee temporelle, necessite une jointure par location et date
print("\najout de stringency_index...")
# creer un subset de covid avec juste location, date et stringency_index
covid_stringency = covid_countries[['location', 'date', 'stringency_index']].dropna(subset=['stringency_index'])

# fusionner avec mpox
mpox_with_stringency = mpox_enriched_final.merge(
    covid_stringency, 
    on=['location', 'date'], 
    how='left',
    suffixes=('', '_covid')
)

if 'stringency_index' not in mpox_enriched_final.columns:
    mpox_enriched_final['stringency_index'] = mpox_with_stringency['stringency_index']
print(f"stringency_index ajoute pour {mpox_enriched_final['stringency_index'].notna().sum()} lignes")

# reproduction_rate ne peut pas etre transfere car c'est specifique a la maladie
print("\nreproduction_rate: specifique a chaque maladie, ne peut pas etre transfere")

# reorganiser les colonnes dans le bon ordre
colonnes_disponibles_mpox = [col for col in colonnes_finales if col in mpox_enriched_final.columns or col == 'continent' or col == 'stringency_index']
colonnes_disponibles_mpox = [col for col in colonnes_disponibles_mpox if col != 'reproduction_rate']
mpox_enriched_final = mpox_enriched_final[colonnes_disponibles_mpox]

# validation finale
print("\n validation finale ")
print(f"covid_countries_final: {covid_countries_final.shape}")
print(f"colonnes: {list(covid_countries_final.columns)}")
print(f"\nmpox_enriched_final: {mpox_enriched_final.shape}")
print(f"colonnes: {list(mpox_enriched_final.columns)}")

# statistiques de completude
print("\n taux de completude des colonnes cles ")
for col in ['population', 'population_density', 'stringency_index', 'mortality_rate']:
    if col in covid_countries_final.columns:
        taux_covid = covid_countries_final[col].notna().sum() / len(covid_countries_final) * 100
        print(f"covid - {col}: {taux_covid:.1f}%")
    if col in mpox_enriched_final.columns:
        taux_mpox = mpox_enriched_final[col].notna().sum() / len(mpox_enriched_final) * 100
        print(f"mpox - {col}: {taux_mpox:.1f}%")

 enrichissement des colonnes manquantes dans mpox 
ajout de la colonne continent...
continent ajoute pour 116057 lignes

ajout de stringency_index...
stringency_index ajoute pour 18035 lignes

reproduction_rate: specifique a chaque maladie, ne peut pas etre transfere

 validation finale 
covid_countries_final: (401502, 20)
colonnes: ['iso_code', 'continent', 'location', 'date', 'new_cases', 'total_cases', 'new_cases_smoothed', 'new_cases_per_million', 'new_cases_smoothed_per_million', 'reproduction_rate', 'new_deaths', 'total_deaths', 'new_deaths_smoothed', 'new_deaths_per_million', 'new_deaths_smoothed_per_million', 'total_deaths_per_million', 'mortality_rate', 'population', 'population_density', 'stringency_index']

mpox_enriched_final: (116057, 19)
colonnes: ['iso_code', 'continent', 'location', 'date', 'new_cases', 'total_cases', 'new_cases_smoothed', 'new_cases_per_million', 'new_cases_smoothed_per_million', 'new_deaths', 'total_deaths', 'new_deaths_smoothed', 'new_deaths_per_mill

In [37]:
# ajout des coordonnees geographiques
print("=== ajout des coordonnees geographiques ===")

# telecharger les donnees de coordonnees
print("telechargement des coordonnees geographiques...")
geo_data = pd.read_csv('https://gist.githubusercontent.com/tadast/8827699/raw/f5cac3d42d16b78348610fc4ec301e9234f82821/countries_codes_and_coordinates.csv')

# nettoyer les noms de colonnes (espaces et guillemets)
geo_data.columns = geo_data.columns.str.strip().str.replace('"', '')

# afficher les colonnes disponibles
print(f"colonnes geo disponibles: {list(geo_data.columns)}")
print(f"nombre de pays avec coordonnees: {len(geo_data)}")

# preparation des donnees geo
geo_subset = geo_data[['Alpha-3 code', 'Latitude (average)', 'Longitude (average)']].copy()
geo_subset.columns = ['iso_code', 'latitude', 'longitude']

# NETTOYER TOUTES LES VALEURS (espaces et guillemets)
print("nettoyage des donnees geographiques...")
geo_subset['iso_code'] = geo_subset['iso_code'].str.strip().str.replace('"', '')
geo_subset['latitude'] = geo_subset['latitude'].str.strip().str.replace('"', '')
geo_subset['longitude'] = geo_subset['longitude'].str.strip().str.replace('"', '')

# convertir latitude et longitude en nombres
geo_subset['latitude'] = pd.to_numeric(geo_subset['latitude'], errors='coerce')
geo_subset['longitude'] = pd.to_numeric(geo_subset['longitude'], errors='coerce')

# fonction pour ajouter les coordonnees a un dataset
def add_coordinates(df, geo_data):
    """ajoute latitude et longitude au dataset"""
    df_with_coords = df.merge(
        geo_data,
        on='iso_code',
        how='left'
    )
    return df_with_coords

# application aux datasets pays seulement
print("\najout des coordonnees aux datasets pays...")
covid_countries_final = add_coordinates(covid_countries_final, geo_subset)
mpox_enriched_final = add_coordinates(mpox_enriched_final, geo_subset)

# preparer les datasets agregations pour la sauvegarde (sans coordonnees)
covid_aggregations_final = covid_aggregations[[col for col in colonnes_finales if col in covid_aggregations.columns]]
mpox_aggregations_final = mpox_aggregations[[col for col in colonnes_finales if col in mpox_aggregations.columns and col != 'reproduction_rate']]

# verification de l'ajout
print("\n=== verification de l'ajout des coordonnees ===")
for name, df in [('covid_countries', covid_countries_final), 
                 ('mpox_countries', mpox_enriched_final)]:
    if 'latitude' in df.columns and 'longitude' in df.columns:
        coords_present = df[['latitude', 'longitude']].notna().all(axis=1).sum()
        nb_locations = df['location'].nunique()
        pays_avec_coords = df.groupby('location')[['latitude', 'longitude']].first().notna().all(axis=1).sum()
        print(f"\n{name}:")
        print(f"  lignes avec coordonnees: {coords_present}/{len(df)} ({coords_present/len(df)*100:.1f}%)")
        print(f"  pays avec coordonnees: {pays_avec_coords}/{nb_locations} ({pays_avec_coords/nb_locations*100:.1f}%)")

print("\nagrégations (sans coordonnees):")
print(f"covid_aggregations: {covid_aggregations_final.shape[0]} lignes")
print(f"mpox_aggregations: {mpox_aggregations_final.shape[0]} lignes")

# exemples de verification
print("\nexemples de coordonnees ajoutees:")
for country in ['France', 'Germany', 'Brazil']:
    exemple = covid_countries_final[covid_countries_final['location'] == country][['location', 'latitude', 'longitude']].drop_duplicates()
    if not exemple.empty:
        print(f"{country}: lat={exemple['latitude'].iloc[0]:.1f}, lon={exemple['longitude'].iloc[0]:.1f}")

# verifier les pays sans coordonnees
pays_sans_coords = covid_countries_final[covid_countries_final['latitude'].isna()]['location'].unique()
if len(pays_sans_coords) > 0:
    print(f"\npays sans coordonnees: {len(pays_sans_coords)}")
    print(f"exemples: {list(pays_sans_coords)[:5]}")

# mise a jour des colonnes finales
colonnes_finales_avec_geo = colonnes_finales + ['latitude', 'longitude']
print(f"\nnombre total de colonnes finales avec coordonnees: {len(colonnes_finales_avec_geo)}")

=== ajout des coordonnees geographiques ===
telechargement des coordonnees geographiques...
colonnes geo disponibles: ['Country', 'Alpha-2 code', 'Alpha-3 code', 'Numeric code', 'Latitude (average)', 'Longitude (average)']
nombre de pays avec coordonnees: 256
nettoyage des donnees geographiques...

ajout des coordonnees aux datasets pays...

=== verification de l'ajout des coordonnees ===

covid_countries:
  lignes avec coordonnees: 405295/421264 (96.2%)
  pays avec coordonnees: 232/243 (95.5%)

mpox_countries:
  lignes avec coordonnees: 119200/121380 (98.2%)
  pays avec coordonnees: 134/137 (97.8%)

agrégations (sans coordonnees):
covid_aggregations: 20163 lignes
mpox_aggregations: 7794 lignes

exemples de coordonnees ajoutees:
France: lat=46.0, lon=2.0
Germany: lat=51.0, lon=9.0
Brazil: lat=-10.0, lon=-55.0

pays sans coordonnees: 11
exemples: ['Bonaire Sint Eustatius and Saba', 'Curacao', 'England', 'Kosovo', 'Northern Cyprus']

nombre total de colonnes finales avec coordonnees: 22


In [38]:
# sauvegarde et validation finale
print(" sauvegarde des datasets nettoyes ")

# sauvegarder les datasets
covid_countries_final.to_csv(OUTPUT_DIR / 'covid_countries_clean.csv', index=False)
covid_aggregations_final.to_csv(OUTPUT_DIR / 'covid_aggregations_clean.csv', index=False)
mpox_enriched_final.to_csv(OUTPUT_DIR / 'mpox_countries_clean.csv', index=False)
mpox_aggregations_final.to_csv(OUTPUT_DIR / 'mpox_aggregations_clean.csv', index=False)

print("fichiers sauvegardes:")
print("  - covid_countries_clean.csv")
print("  - covid_aggregations_clean.csv")
print("  - mpox_countries_clean.csv")
print("  - mpox_aggregations_clean.csv")

# resume final de l'etl
print("\nresume final de l'etl")
print(f"\ncovid original: {covid_raw.shape[0]} lignes -> covid pays: {covid_countries_final.shape[0]} + agregations: {covid_aggregations_final.shape[0]}")
print(f"mpox original: {mpox_raw.shape[0]} lignes -> mpox pays: {mpox_enriched_final.shape[0]} + agregations: {mpox_aggregations_final.shape[0]}")

print("\noperations effectuees:")
print("  1. conversion dates en datetime")
print("  2. fusion des doublons (7770 paires dans covid)")
print("  3. correction valeurs cumulatives decroissantes (cummax)")
print("  4. suppression lignes toutes nan (1 dans mpox)")
print("  5. correction reproduction_rate negatif (178 valeurs)")
print("  6. calcul mortality_rate avec plafond a 100%")
print("  7. separation pays vs agregations")
print("  8. enrichissement mpox avec donnees covid")
print("  9. selection colonnes finales (22 pour covid, 21 pour mpox)")
print(" 10. ajout coordonnees geographiques")

# qualite des donnees finales detaillee
print("\n qualite des donnees finales ")
for name, df in [('covid_countries', covid_countries_final), ('mpox_countries', mpox_enriched_final)]:
    print(f"\n{name}:")
    print(f"  dimensions: {df.shape}")
    print(f"  plage dates: {df['date'].min()} a {df['date'].max()}")
    print(f"  pays uniques: {df['location'].nunique()}")
    print(f"  taux global de completude: {100 - (df.isnull().sum().sum() / (df.shape[0] * df.shape[1]) * 100):.1f}%")
    
    # verification des coordonnees
    if 'latitude' in df.columns and 'longitude' in df.columns:
        coords_complete = df[['latitude', 'longitude']].notna().all(axis=1).sum()
        pays_avec_coords = df.groupby('location')[['latitude', 'longitude']].first().notna().all(axis=1).sum()
        print(f"  coordonnees: {coords_complete}/{len(df)} lignes ({coords_complete/len(df)*100:.1f}%)")
        print(f"  pays avec coordonnees: {pays_avec_coords}/{df['location'].nunique()} ({pays_avec_coords/df['location'].nunique()*100:.1f}%)")

# taux de completude par colonne importante
print("\n completude des colonnes cles ")
colonnes_cles = ['population', 'population_density', 'stringency_index', 'mortality_rate', 'latitude', 'longitude']

for dataset_name, df in [('covid_countries', covid_countries_final), ('mpox_countries', mpox_enriched_final)]:
    print(f"\n{dataset_name}:")
    for col in colonnes_cles:
        if col in df.columns:
            taux = df[col].notna().sum() / len(df) * 100
            print(f"  {col}: {taux:.1f}%")
        else:
            print(f"  {col}: colonne absente")

# verification finale des fichiers
print("\n verification taille des fichiers ")
for filename in ['covid_countries_clean.csv', 'covid_aggregations_clean.csv', 
                 'mpox_countries_clean.csv', 'mpox_aggregations_clean.csv']:
    if os.path.exists(filename):
        size_mb = os.path.getsize(filename) / (1024 * 1024)
        print(f"{filename}: {size_mb:.1f} MB")

print("\netl termine avec succes!")

 sauvegarde des datasets nettoyes 
fichiers sauvegardes:
  - covid_countries_clean.csv
  - covid_aggregations_clean.csv
  - mpox_countries_clean.csv
  - mpox_aggregations_clean.csv

resume final de l'etl

covid original: 429435 lignes -> covid pays: 421264 + agregations: 20163
mpox original: 123852 lignes -> mpox pays: 121380 + agregations: 7794

operations effectuees:
  1. conversion dates en datetime
  2. fusion des doublons (7770 paires dans covid)
  3. correction valeurs cumulatives decroissantes (cummax)
  4. suppression lignes toutes nan (1 dans mpox)
  5. correction reproduction_rate negatif (178 valeurs)
  6. calcul mortality_rate avec plafond a 100%
  7. separation pays vs agregations
  8. enrichissement mpox avec donnees covid
  9. selection colonnes finales (22 pour covid, 21 pour mpox)
 10. ajout coordonnees geographiques

 qualite des donnees finales 

covid_countries:
  dimensions: (421264, 22)
  plage dates: 2020-01-01 00:00:00 a 2024-08-14 00:00:00
  pays uniques: 243
 